In [89]:
import numpy as np
import random
import scipy
import scipy.io
import pandas as pd
from pathlib import Path
import os
import collections
from natsort import natsorted
import json
import pickle
import warnings
import sys
from numpy import interp
from pprint import pprint

from numpy.testing import assert_array_equal

warnings.filterwarnings("ignore")

from sklearn.preprocessing import OrdinalEncoder, LabelBinarizer, LabelEncoder
from sklearn.multiclass import OneVsRestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import calibration_curve
from sklearn.metrics import (
    brier_score_loss,
    roc_curve,
    average_precision_score,
    roc_auc_score,
    f1_score,
    recall_score,
    jaccard_score,
    balanced_accuracy_score,
    accuracy_score,
    auc,
    precision_score,
    plot_precision_recall_curve,
    average_precision_score,
    precision_recall_curve,
    confusion_matrix,
    cohen_kappa_score,
    make_scorer,
    precision_recall_fscore_support,
)
from sklearn.inspection import permutation_importance
from sklearn.model_selection import (
    StratifiedGroupKFold,
    cross_validate,
    StratifiedShuffleSplit,
    LeaveOneGroupOut,
)
from sklearn.utils import resample
import sklearn
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline

import mne
from mne.time_frequency import read_tfrs

mne.set_log_level("ERROR")
from mne_bids import BIDSPath, get_entities_from_fname, get_entity_vals, read_raw_bids

from eztrack.io import read_derivative_npy

sys.path.append("../../")
from episcalp.scripts.spikes.summary import _get_spike_annots
from episcalp.cross_validate import (
    get_X_features,
    load_derived_datasets,
    _compute_spike_rates,
    exclude_subjects
)
from episcalp.preprocess.montage import _standard_lobes
from episcalp.utils.utils import NumpyEncoder

# if you installed sporf via README
#from oblique_forests.sporf import ObliqueForestClassifier

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define possible helper functions

In [2]:
def _get_exp_condition(subject, root):
    part_fname = os.path.join(root, "participants.tsv")
    df = pd.read_csv(part_fname, sep="\t")

    if not subject.startswith("sub-"):
        subject = f"sub-{subject}"

    return df[df["participant_id"] == subject]

In [10]:
def convert_experimental_cond_to_y(experimental_condition_list):
    """Encoder for y labels."""
    # Group name keys, assigned y-label values
    experimental_condition_map = {
        "non-epilepsy-normal-eeg": 0,
        "epilepsy-normal-eeg": 1,
        "epilepsy-abnormal-eeg": 1,
    }
    return [experimental_condition_map[cond] for cond in experimental_condition_list]

In [11]:
def _preprocess_epochs_tfr(data):
    """Turn TFR data into a 2D array."""
    assert data.ndim == 4

    # take the average over frequencies
    data = np.mean(data, axis=2)

    # move the epoch ("window") axis to last
    data = np.moveaxis(data, 0, -2)

    # compress the time axis
    data = np.mean(data, axis=-1)

    # convert to dB
    data = 20 * np.log10(data)

    data = np.reshape(data, (data.shape[0], -1))
    return data

In [12]:
def _load_subject_derivs(deriv_path, read_func, subjects=None, search_str="*.npy"):
    if subjects is None:
        subjects = get_entity_vals(deriv_path, "subject")

    print(f"Loading data for subjects: {subjects}")
    print(deriv_path)

    this_path = deriv_path
    while this_path.parent.name != "derivatives" and this_path.parent.name != "/":
        this_path = this_path.parent
    root = this_path.parent.parent

    dataset = collections.defaultdict(list)
    for subject in subjects:
        subj_path = deriv_path / f"sub-{subject}"

        # get all files of certain search_str
        fpaths = subj_path.glob(search_str)

        # now load in all file paths
        for fpath in fpaths:
            deriv = read_func(fpath, source_check=False)

            # extract data
            ch_names = deriv.ch_names
            deriv_data = deriv.get_data()

            dataset["subject"].append(subject)
            dataset["data"].append(deriv_data)
            dataset["ch_names"].append(ch_names)
            dataset["roots"].append(root)

    return dataset

In [13]:
def _load_subject_tfrs(deriv_path, read_func, subjects=None, search_str="*.h5"):
    if subjects is None:
        subjects = get_entity_vals(deriv_path, "subject")

    print(f"Loading data for subjects: {subjects}")
    print(deriv_path)

    this_path = deriv_path
    while this_path.parent.name != "derivatives" and this_path.parent.name != "/":
        this_path = this_path.parent
    root = this_path.parent.parent

    dataset = collections.defaultdict(list)
    for subject in subjects:
        subj_path = deriv_path / f"sub-{subject}"

        # get all files of certain search_str
        fpaths = subj_path.glob(search_str)

        # now load in all file paths
        for fpath in fpaths:
            try:
                deriv = read_func(fpath)[0]
            except Exception as e:
                print(f"Failed trying to load {fpath}.")
                raise Exception(e)
            # extract data
            ch_names = deriv.ch_names
            deriv_data = deriv.data

            # apply mean across axis 2
            deriv_data = _preprocess_epochs_tfr(deriv_data)
            #             deriv_data = np.mean(deriv_data, axis=2)

            dataset["subject"].append(subject)
            dataset["data"].append(deriv_data)
            dataset["ch_names"].append(ch_names)
            dataset["roots"].append(root)

    return dataset

# Define data directories

In [14]:
user = "patrick"
if user == "patrick":
    jhroot = Path("D:/OneDriveParent/OneDrive - Johns Hopkins/Shared Documents/bids")
    jeffroot = Path("D:/OneDriveParent/Johns Hopkins/Jefferson_Scalp - Documents/root")

    # not ready yet
    upmcroot = Path("/Users/adam2392/Johns Hopkins/UPMC_Scalp - Documents/")
elif user == "adam":
    jhroot = Path("/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/")
    jeffroot = Path("/Users/adam2392/Johns Hopkins/Jefferson_Scalp - Documents/root/")

    # not ready yet
    upmcroot = Path("/Users/adam2392/Johns Hopkins/UPMC_Scalp - Documents/")

In [15]:
reference = "monopolar"
radius = "1.25"
winsize = "500"
stepsize = "250"

# define derivative chains
ss_deriv_chain = Path("sourcesink") /  f"win-{winsize}" / f"step-{stepsize}" / reference
frag_deriv_chain = Path("fragility") / f"radius{radius}" / f"win-{winsize}" / f"step-{stepsize}" / reference

delta_tfr_deriv_chain = Path("tfr") / "delta"
theta_tfr_deriv_chain = Path("tfr") / "theta"
alpha_tfr_deriv_chain = Path("tfr") / "alpha"
beta_tfr_deriv_chain = Path("tfr") / "beta"

In [16]:
if user == "adam":
    root = Path("/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/")
elif user == "patrick":
    root = Path("D:/OneDriveParent/OneDrive - Johns Hopkins/Shared Documents/bids")
deriv_root = root / "derivatives"
figure_dir = deriv_root
source_root = root / "sourcedata"

# Load in Datasets (Once)

In [17]:
roots = [jhroot, jeffroot]
deriv_chains = [frag_deriv_chain, frag_deriv_chain]

load_func = _load_subject_derivs
read_func = read_derivative_npy
search_str = "*desc-perturbmatrix*.npy"

# load fragility dataset
fragility_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)
X_fragility = get_X_features(fragility_dataset, data_name="data", feature_names=['quantiles'])

Loading data for subjects: ['jhh001', 'jhh002', 'jhh003', 'jhh004', 'jhh005', 'jhh006', 'jhh007', 'jhh008', 'jhh009', 'jhh010', 'jhh011', 'jhh012', 'jhh013', 'jhh014', 'jhh015', 'jhh016', 'jhh017', 'jhh018', 'jhh019', 'jhh020', 'jhh021', 'jhh022', 'jhh023', 'jhh024', 'jhh025', 'jhh026', 'jhh027', 'jhh028', 'jhh029', 'jhh030', 'jhh101', 'jhh102', 'jhh103', 'jhh104', 'jhh105', 'jhh106', 'jhh107', 'jhh108', 'jhh109', 'jhh110', 'jhh111', 'jhh112', 'jhh113', 'jhh114', 'jhh115', 'jhh116', 'jhh117', 'jhh118', 'jhh119', 'jhh120', 'jhh121', 'jhh122', 'jhh124', 'jhh125', 'jhh126', 'jhh127', 'jhh128', 'jhh201', 'jhh202', 'jhh203', 'jhh204', 'jhh205', 'jhh206', 'jhh207', 'jhh208', 'jhh209', 'jhh210', 'jhh211', 'jhh212', 'jhh213', 'jhh214', 'jhh215', 'jhh216', 'jhh217', 'jhh218', 'jhh219', 'jhh220', 'jhh221', 'jhh222', 'jhh223', 'jhh224', 'jhh225', 'jhh226', 'jhh227', 'jhh228', 'jhh229']
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-2

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log' -> 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log.1'
Call stack:
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File

D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jhh001\sub-jhh001_run-01_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jhh001', 'session': None, 'task': None, 'acquisition': None, 'run': '01', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jhh002\sub-jhh002_run-01_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jhh002', 'session': None, 'task': None, 'acquisition': None, 'run': '01', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jhh003\sub-jhh003_run-01_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jhh003', 'session': None, 'task': None, 'acquisition': None, 'run': '01', 'pro

  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\patri\.virtualenvs\episcalp-235KLvvi\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\patri\.virtualenvs\episcalp-235KLvvi\lib\site-packages\traitlets\config\application.py", line 845, in launch_instance
    app.start()
  File "C:\Users\patri\.virtualenvs\episcalp-235KLvvi\lib\site-packages\ipykernel\kernelapp.py", line 612, in start
    self.io_loop.start()
  File "C:\Users\patri\.virtualenvs\episcalp-235KLvvi\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "c:\users\patri\appdata\local\programs\python\python38\lib\asyncio\windows_events.py", line 316, in run_forever
 

D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jhh007\sub-jhh007_run-01_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jhh007', 'session': None, 'task': None, 'acquisition': None, 'run': '01', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jhh008\sub-jhh008_run-01_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jhh008', 'session': None, 'task': None, 'acquisition': None, 'run': '01', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\OneDrive - Johns Hopkins\Shared Documents\bids\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jhh009\sub-jhh009_run-01_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jhh009', 'session': None, 'task': None, 'acquisition': None, 'run': '01', 'pro

...

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log' -> 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log.1'
Call stack:
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File

D:\OneDriveParent\Johns Hopkins\Jefferson_Scalp - Documents\root\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jeff202\sub-jeff202_ses-1_run-4_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jeff202', 'session': '1', 'task': None, 'acquisition': None, 'run': '4', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\Johns Hopkins\Jefferson_Scalp - Documents\root\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jeff202\sub-jeff202_ses-1_run-5_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jeff202', 'session': '1', 'task': None, 'acquisition': None, 'run': '5', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\Johns Hopkins\Jefferson_Scalp - Documents\root\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jeff203\sub-jeff203_ses-1_run-1_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jeff203', 'session': '1', 'task': None, 'acquisition': None

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log' -> 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log.1'
Call stack:
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File

D:\OneDriveParent\Johns Hopkins\Jefferson_Scalp - Documents\root\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jeff206\sub-jeff206_ses-1_run-1_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jeff206', 'session': '1', 'task': None, 'acquisition': None, 'run': '1', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\Johns Hopkins\Jefferson_Scalp - Documents\root\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jeff207\sub-jeff207_ses-1_run-1_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jeff207', 'session': '1', 'task': None, 'acquisition': None, 'run': '1', 'processing': None, 'space': None, 'recording': None, 'split': None}
D:\OneDriveParent\Johns Hopkins\Jefferson_Scalp - Documents\root\derivatives\fragility\radius1.25\win-500\step-250\monopolar\sub-jeff208\sub-jeff208_ses-1_run-1_desc-perturbmatrix_eeg.json
source entities: {'subject': 'jeff208', 'session': '1', 'task': None, 'acquisition': None

--- Logging error ---
Traceback (most recent call last):
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 70, in emit
    self.doRollover()
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 171, in doRollover
    self.rotate(self.baseFilename, dfn)
  File "c:\users\patri\appdata\local\programs\python\python38\lib\logging\handlers.py", line 111, in rotate
    os.rename(source, dest)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log' -> 'd:\\desktop\\eztrack\\.eztrack\\logging\\eztrack.log.1'
Call stack:
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\users\patri\appdata\local\programs\python\python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File

In [ ]:
roots = [jhroot, jeffroot]
deriv_chains = [ss_deriv_chain, ss_deriv_chain]

load_func = _load_subject_derivs
read_func = read_derivative_npy
search_str = "*desc-ssindmatrix*.npy"

# load fragility dataset
ss_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)
X_ss = get_X_features(ss_dataset, data_name="data", feature_names=['quantiles'])

In [ ]:
roots = [jhroot, jeffroot]
deriv_chains = [ss_deriv_chain, ss_deriv_chain]

load_func = _load_subject_derivs
read_func = read_derivative_npy
search_str = "*desc-sinkindmatrix*.npy"

# load fragility dataset
sink_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)
X_sink = get_X_features(sink_dataset, data_name="data", feature_names=['quantiles'])

In [ ]:
roots = [jhroot, jeffroot]
deriv_chains = [delta_tfr_deriv_chain, delta_tfr_deriv_chain]

load_func = _load_subject_tfrs
read_func = read_tfrs
search_str = "*.h5"

# load fragility dataset
delta_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)
X_delta = get_X_features(delta_dataset, data_name="data", feature_names=['quantiles'])

In [ ]:
roots = [jhroot, jeffroot]
deriv_chains = [theta_tfr_deriv_chain, theta_tfr_deriv_chain]

load_func = _load_subject_tfrs
read_func = read_tfrs
search_str = "*.h5"

# load fragility dataset
theta_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)

X_theta = get_X_features(theta_dataset, data_name="data", feature_names=['quantiles'])

In [ ]:
roots = [jhroot, jeffroot]
deriv_chains = [alpha_tfr_deriv_chain, alpha_tfr_deriv_chain]

load_func = _load_subject_tfrs
read_func = read_tfrs
search_str = "*.h5"

# load fragility dataset
alpha_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)

X_alpha = get_X_features(alpha_dataset, data_name="data", feature_names=['quantiles'])

In [ ]:
roots = [jhroot, jeffroot]
deriv_chains = [beta_tfr_deriv_chain, beta_tfr_deriv_chain]

load_func = _load_subject_tfrs
read_func = read_tfrs
search_str = "*.h5"

# load fragility dataset
beta_dataset = load_derived_datasets(
    roots,
    deriv_chains=deriv_chains,
    load_func=load_func,
    read_func=read_func,
    search_str=search_str,
)
X_beta = get_X_features(beta_dataset, data_name="data", feature_names=['quantiles'])

In [19]:
feature_dict = {
    "fragility": X_fragility,
    #"sourcesink": X_ss,
    #"sink": X_sink,
    #"delta": X_delta,
    #"theta": X_theta,
    #"alpha": X_alpha,
    #"beta": X_beta,
}

In [20]:
subjects = np.array(fragility_dataset["subject"])
roots = fragility_dataset["roots"]

exp_conditions = []
for subject, root in zip(subjects, roots):
    subj_df = _get_exp_condition(subject, root)
    exp_condition = subj_df["exp_condition"].values[0]
    exp_conditions.append(exp_condition)

# Define run specific parameters

In [80]:
features = ["fragility"]
n_splits = 20
train_size = 0.7
random_state = 12345
clf_name = 'rf'

cv = StratifiedShuffleSplit(
    n_splits=n_splits,
    train_size=train_size,
    random_state=random_state,
)

# Define input X and labels Y

In [22]:
X = None
for fdx, feat in enumerate(features):
    if fdx == 0:
        X = feature_dict[feat]
    else:
        X = np.hstack(
            (
                X,
                feature_dict[feat]
            )
        )

In [23]:
y = np.array(convert_experimental_cond_to_y(exp_conditions))
y_classes = ['non-epilepsy', 'epilepsy']

In [91]:
# Further subset the subjects if desired
exclusion_criteria = {
    "exp_condition": ["epilepsy-abnormal-eeg"],
    "final_diagnosis": None,
    "epilepsy_type": ["generalized"],
    "epilepsy_hemisphere": None,
    "epilepsy_lobe": None,
}
X, y, keep_subjects = exclude_subjects(X, y, subjects, roots, exclusion_criteria)
print(X.shape, y.shape, keep_subjects.shape)

(70, 5) (70,) (70,)


# Define Classification Model

In [92]:
max_features = X.shape[1]
rf_model_params = {
    "n_estimators": 1000,
    "max_features": max_features,
    "n_jobs": -1,
    "random_state": random_state,
}
lr_model_params = {
    "n_jobs": -1,
    "random_state": random_state,
    #                    "penalty": "l2"
}

In [93]:
if clf_name == "rf":
    clf = RandomForestClassifier(**rf_model_params)
elif clf_name == "sporf":
    # only used if you installed cysporf
    clf = ObliqueForestClassifier(**rf_model_params)
elif clf_name == "lr":
    clf = LogisticRegression(**lr_model_params)
    clf = make_pipeline(
        StandardScaler(),
        clf
    )

# Define Cross Validation Experiment

In [94]:
scoring_funcs = {
    "balanced_accuracy": make_scorer(balanced_accuracy_score),
    "cohen_kappa_score": make_scorer(cohen_kappa_score),
    "roc_auc": "roc_auc",  # roc_auc_score,
    "f1": "f1",  # f1_score,
    "recall": "recall",  # makerecall_score,
    "precision": "precision",  # precision_score,
    "jaccard": "jaccard",  # jaccard_score,
    "average_precision": "average_precision",  # average_precision_score,
    "neg_brier_score": "neg_brier_score",  # brier_score_loss,
}

# define multiclass cross_validation scoring metrics
f1_micro = lambda x: make_scorer(f1_score, average="micro")
recall_micro = lambda x: make_scorer(recall_score, average="micro")
precision_micro = lambda x: make_scorer(precision_score, average="micro")
jaccard_micro = lambda x: make_scorer(jaccard_score, average="micro")
roc_ovr = lambda x: make_scorer(roc_auc_score, multi_class="ovr")

# some scorers need to get defined with this for multiclass
def cohen_scorer(clf, X, y):
    y_pred = clf.predict(X).argmax(axis=1)
    y = y.argmax(axis=1)
    return cohen_kappa_score(y, y_pred)


def balanced_accuracy_scorer(clf, X, y):
    y_pred = clf.predict(X).argmax(axis=1)
    y = y.argmax(axis=1)
    return balanced_accuracy_score(y, y_pred)


multiclass_scoring_funcs = {
    "balanced_accuracy": balanced_accuracy_scorer,
    "cohen_kappa_score": cohen_scorer,
    "roc_auc_ovr": "roc_auc_ovr",  # roc_ovr,
    "f1_micro": "f1_micro",  # f1_micro,
    "recall_micro": "recall_micro",
    "precision_micro": "precision_micro",
    "jaccard_micro": "jaccard_micro",
}

if len(np.unique(y_classes)) == 2:
    scoring = scoring_funcs
else:
    scoring = multiclass_scoring_funcs

print(scoring)

{'balanced_accuracy': make_scorer(balanced_accuracy_score), 'cohen_kappa_score': make_scorer(cohen_kappa_score), 'roc_auc': 'roc_auc', 'f1': 'f1', 'recall': 'recall', 'precision': 'precision', 'jaccard': 'jaccard', 'average_precision': 'average_precision', 'neg_brier_score': 'neg_brier_score'}


In [96]:
# run cross-validation
scores = cross_validate(
    clf,
    X,
    y,
    groups=keep_subjects,
    cv=cv,
    scoring=scoring,
    return_estimator=True,
    return_train_score=True,
    n_jobs=-1,
    error_score="raise",
)

# get the estimators
estimators = scores.pop("estimator")

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
print("Using estimator ", estimators[0])

# initialize keys to list
for key in [
    "test_fraction_pos",
    "test_mean_pred_value",
    "test_fpr",
    "test_tpr",
    "test_fnr",
    "test_tnr",
    "test_thresholds",
    "train_fraction_pos",
    "train_mean_pred_value",
    "train_y_pred_proba",
    "train_fpr",
    "train_tpr",
    "train_fnr",
    "train_tnr",
    "train_thresholds",
    "train_subjects",
    "test_subjects",
    "test_y_pred_proba",
    "test_importances",
    "test_specificity",
    "test_y",
    "test_y_encodings",
]:
    scores[key] = []

for idx, (train_inds, test_inds) in enumerate(cv.split(X, y)):
    estimator = estimators[idx]

    # append subjects
    scores["train_subjects"].append(subjects[train_inds])
    scores["test_subjects"].append(subjects[test_inds])
    scores["test_y"].append(y[test_inds])
    #     scores["test_y_encodings"].append(y_encode[test_inds])

    # evaluate on the train set
    y_train = y[train_inds]
    X_train = X[train_inds, :]
    y_pred_prob = estimator.predict_proba(X_train)
    y_pred_label = np.argmax(y_pred_prob, axis=1)

    # compute calibration curve
    if len(np.unique(y_classes)) == 2:
        if y_pred_prob.ndim == 2:
            y_pred_prob = y_pred_prob[:, -1]
        fraction_of_positives, mean_predicted_value = calibration_curve(
            y_train, y_pred_prob, n_bins=10, strategy="quantile"
        )
        scores["train_fraction_pos"].append(fraction_of_positives)
        scores["train_mean_pred_value"].append(mean_predicted_value)

    # append training data
    scores["train_y_pred_proba"].append(y_pred_prob)

    # store ROC curve metrics on the held-out test set
    if len(np.unique(y_classes)) == 2:
        fpr, tpr, thresholds = roc_curve(y_train, y_pred_prob, pos_label=1)
        fnr, tnr, neg_thresholds = roc_curve(y_train, y_pred_prob, pos_label=0)
        scores["train_fpr"].append(fpr)
        scores["train_tpr"].append(tpr)
        scores["train_fnr"].append(fnr)
        scores["train_tnr"].append(tnr)
        scores["train_thresholds"].append(thresholds)

    # evaluate on the test set
    y_test = y[test_inds]
    X_test = X[test_inds, :]
    y_pred_prob = estimator.predict_proba(X_test)
    y_pred_label = np.argmax(y_pred_prob, axis=1)

    # compute calibration curve
    if len(np.unique(y_classes)) == 2:
        if y_pred_prob.ndim == 2:
            y_pred_prob = y_pred_prob[:, -1]
        fraction_of_positives, mean_predicted_value = calibration_curve(
            y_test, y_pred_prob, n_bins=10, strategy="quantile"
        )
        scores["test_fraction_pos"].append(fraction_of_positives)
        scores["test_mean_pred_value"].append(mean_predicted_value)

    # append testing data
    scores["test_y_pred_proba"].append(y_pred_prob)

    # store ROC curve metrics on the held-out test set
    if len(np.unique(y_classes)) == 2:
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob, pos_label=1)
        fnr, tnr, neg_thresholds = roc_curve(y_test, y_pred_prob, pos_label=0)
        scores["test_fpr"].append(fpr)
        scores["test_tpr"].append(tpr)
        scores["test_fnr"].append(fnr)
        scores["test_tnr"].append(tnr)
        scores["test_thresholds"].append(thresholds)

    if y_test.ndim == 2:
        y_test_ = y_test.astype(int).argmax(axis=1)
    else:
        y_test_ = y_test.astype(int)
    conf_matrix = confusion_matrix(y_test_, (y_pred_label).astype(int))
    if len(np.unique(y_classes)) == 2:
        tn, fp, fn, tp = conf_matrix.ravel()
        scores["test_specificity"].append(tn / (tn + fp))

    # get the permutation importances
    r = permutation_importance(
        estimator,
        X_test,
        y_test,
        n_repeats=5,
        random_state=random_state,
        n_jobs=-1,
        scoring="roc_auc_ovr",
    )
    scores["test_importances"].append(r.importances)

In [ ]:
# add metadata
scores["y_classes"] = y_classes

In [ ]:
exp_name =  "-".join(features) + "distribution"
fname = (
    deriv_root.parent
    / "derivatives"
    / f"cv{n_splits}_{clf_name}_mtry{max_features}_{exp_name}_withstd.json"
)
print(fname)